### Checks which sequences have been downloaded in folder `wd`

In [6]:
import sys
sys.path.append('/home/grg/git/xnat-monitor/')
import bbrc
from bbrc import xnat
import string, os
import os.path as osp
import numpy as np
from glob import glob

In [7]:
central = xnat.connect_xnat('/home/grg/.xnat_bsc.cfg')
#e = list(central.select.project('ALFA_PLUS').subject('66237').experiments())[0]
#xnat.check_session(central, e.label(), send_mail=False)

In [21]:
wd = '/tmp/downloads/NIFTI'
sequences = ['1501', '1601', '1701', '1301', '1401', '1201', '1101', '301', '401', '501', '701', '901']
subjects = [string.atoi(e) for e in open('/tmp/amyloid_subjects.txt').read().split('\n') if e != '']
bvec_correct =  [10162, 10515, 10627, 10649, 10711, 10949, 11262, 11514, 11745, 12279, 12741, 12991, 13035, 13105, 13107, 13193, 13259,
                 21136, 21138, 44008, 44301, 44396, 55854, 66008, 66027, 66100, 66162, 66301, 77012, 77164]
subjects = [e for e in subjects if not e in bvec_correct]

In [27]:
t = []
for subject in subjects:
    t.append([subject])
    n = 0
    for seq in sequences:
        fp = osp.join(wd, '%s_%s.nii.gz'%(subject, seq))
        t[-1].append(osp.isfile(fp))
        n = n + osp.isfile(fp)
    t[-1].append(n)
        
import pandas as pd
df = pd.DataFrame(t, columns=['subject', '1501', '1601', '1701', '1301', '1401', '1201', '1101', '301', '401', '501', '701', '901', 'n'])
duplicates = df[df['n']!=2]['subject'].tolist()
for each in duplicates:
    print each
    expids = xnat.subject_experiments(central, 'ALFA_PLUS', str(each))
    if len(expids) == 0: expids = xnat.subject_experiments(central, 'ALFA_OPCIONAL', str(each))
    print expids
    e = expids[0] if not each in [12627, 44329] else expids[1]
    d = xnat.series_desc(central, e.id())
    seq = list(np.array(sequences)[df[df['subject']==each].iloc[0].tolist()[1:-1]])
    print [(s, d[s]) for s in seq]
        

10019
[<Experiment Object> BBRC_E00046]
[('1301', 'DWI_ALFA1')]
10044
[<Experiment Object> BBRC_E01935]
[('1301', 'DWI_ALFA1')]
10065
[<Experiment Object> BBRC_E00300]
[('1601', 'DWI_ALFA1')]
10070
[<Experiment Object> BBRC_E01500]
[('1301', 'DWI_ALFA1')]
10137
[<Experiment Object> BBRC_E01055]
[('1501', 'DWI_ALFA1')]
10200
[<Experiment Object> BBRC_E00108]
[('1301', 'DWI_ALFA1')]
10225
[<Experiment Object> BBRC_E00087]
[('1301', 'DWI_ALFA1')]
10235
[<Experiment Object> BBRC_E02019]
[('1701', 'DWI_ALFA1')]
10245
[<Experiment Object> BBRC_E01378]
[('1401', 'DWI_ALFA1')]
10319
[<Experiment Object> BBRC_E00052]
[('1301', 'DWI_ALFA1')]
10329
[<Experiment Object> BBRC_E00089]
[('1301', 'DWI_ALFA1')]
10424
[<Experiment Object> BBRC_E01025]
[('1601', 'DWI_ALFA1')]
10560
[<Experiment Object> BBRC_E02015]
[('1301', 'DWI_ALFA1')]
10613
[<Experiment Object> BBRC_E00295]


KeyError: '1301'

### Subjects with wrong numbers of images

In [33]:
df[df['n']!=2]

,subject,1501,1601,1701,1301,1401,1201,1101,301,401,501,701,901,n
7,10225,False,False,False,True,True,False,False,False,True,False,False,False,3
35,11273,False,False,False,False,True,False,False,False,True,False,False,True,3
40,11514,True,True,False,False,False,False,False,False,True,False,False,False,3
41,11540,False,False,False,True,False,False,False,True,True,False,False,False,3
63,12991,True,False,False,False,True,False,False,False,True,False,False,False,3
69,13166,False,False,False,True,True,False,False,False,True,False,False,False,3
84,44363,False,False,False,True,True,False,False,True,False,False,False,False,3
87,55038,False,False,False,True,True,False,False,False,True,False,False,False,3
94,55854,False,False,False,False,True,True,False,True,False,False,False,False,3
100,66162,True,False,False,False,True,False,False,False,True,False,False,False,3


### Creates a clean tree with files

In [34]:
wd = '/tmp/downloads/NIFTI'
wd2 = '/tmp/AmylStaging/'

import os
for i, s in enumerate(subjects):
    print s
    files = [e.split('_')[1].split('.')[0] for e in glob(osp.join(wd, '%s_*'%s)) if not e.endswith('.json')]
    count = [files.count(e) for e in set(files)]
    d = dict(zip(count, set(files)))
    #dtiseq, t1seq = d[3], d[1]
    dtiseq = d[3]
    #t1fp = osp.join(wd, '%s_%s.nii.gz'%(s, t1seq))
    dwi, bval, bvec = [osp.join(wd, '%s_%s.%s'%(s, dtiseq, e)) for e in ['nii.gz', 'bval', 'bvec']]
    print s, d, dwi, bval, bvec
    subd = osp.join(wd2, str(s))
    commands = []
    commands.append('mkdir %s'%subd)
    #commands.append('mkdir %s'%osp.join(subd, 'T1'))
    commands.append('mkdir %s'%osp.join(subd, 'DWI'))
    
    #cmd = 'cp %s %s'%(t1fp, osp.join(wd2, str(s), 'T1', '%s.nii.gz'%s))
    #commands.append(cmd)
    cmd = 'cp %s %s'%(dwi, osp.join(wd2, str(s), 'DWI', '%s.nii.gz'%s))
    commands.append(cmd)
    cmd = 'cp %s %s'%(bval, osp.join(wd2, str(s), 'DWI', '%s.bval'%s))
    commands.append(cmd)
    cmd = 'cp %s %s'%(bvec, osp.join(wd2, str(s), 'DWI', '%s.bvec'%s))
    commands.append(cmd)
    
    #print commands
    for cmd in commands:
        print cmd
        os.system(cmd)
    
    
    

10019
10019 {3: '1301'} /tmp/downloads/NIFTI/10019_1301.nii.gz /tmp/downloads/NIFTI/10019_1301.bval /tmp/downloads/NIFTI/10019_1301.bvec
mkdir /tmp/AmylStaging/10019
mkdir /tmp/AmylStaging/10019/DWI
cp /tmp/downloads/NIFTI/10019_1301.nii.gz /tmp/AmylStaging/10019/DWI/10019.nii.gz
cp /tmp/downloads/NIFTI/10019_1301.bval /tmp/AmylStaging/10019/DWI/10019.bval
cp /tmp/downloads/NIFTI/10019_1301.bvec /tmp/AmylStaging/10019/DWI/10019.bvec
10044
10044 {3: '1301'} /tmp/downloads/NIFTI/10044_1301.nii.gz /tmp/downloads/NIFTI/10044_1301.bval /tmp/downloads/NIFTI/10044_1301.bvec
mkdir /tmp/AmylStaging/10044
mkdir /tmp/AmylStaging/10044/DWI
cp /tmp/downloads/NIFTI/10044_1301.nii.gz /tmp/AmylStaging/10044/DWI/10044.nii.gz
cp /tmp/downloads/NIFTI/10044_1301.bval /tmp/AmylStaging/10044/DWI/10044.bval
cp /tmp/downloads/NIFTI/10044_1301.bvec /tmp/AmylStaging/10044/DWI/10044.bvec
10065
10065 {3: '1601'} /tmp/downloads/NIFTI/10065_1601.nii.gz /tmp/downloads/NIFTI/10065_1601.bval /tmp/downloads/NIFTI/10065

In [30]:
d

{3: '1301'}

### Building commands for denoising

In [1]:
import sys 
sys.path.append('/home/grg/git/thesaurus/')
import thesaurus as th
jf = '/home/grg/git/thesaurus/alfa_dwi_pipeline_aug2016.json'
dd = '/home/grg/data/AmylStaging'
json_io = '/home/grg/git/thesaurus/alfa_dwi_pipeline_io_aug2016.json'
a = th.ALFAHelper(directory=dd, jsonfile=json_io)

from glob import glob
subjects = [e for e in open('/tmp/denoising_list_part2.txt').read().split('\n') if e != '']

cmds = []
names = []
for e in ['denoising']:
    for s in subjects:  
        print s
        names.append(s)
        cmd = a.parse_command(str(s), e, jsonfile=jf)
        cmds.append(cmd)
 
# Writing commands in a textfile
fp1 = '/tmp/denoising.sh'
fp2 = '/tmp/denoising.workflow'
w = open(fp1, 'w')
for e in cmds:
    w.write('%s\n'%e)
w.close()

import sys
sys.path.append('/home/grg/git/alfa')
import create_workflow as cw

cw.create_workflow(fp1, fp2, names=names, verbose=True)

Loading toolbox baby
Loading toolbox brainrat
Loading toolbox connectomist
Loading toolbox cortical_surface
Loading toolbox data management
Loading toolbox datamind
Loading toolbox freesurfer
Loading toolbox morphologist
Loading toolbox nuclearimaging
Loading toolbox spm
Loading toolbox structural_analysis
Loading toolbox tms
Loading toolbox tools
Loading toolbox viewers
Loading toolbox My processes
The log file for this session is '/home/grg/.brainvisa/brainvisa24.log'
Database /home/grg/data/AmylBV must be updated because the database file is too
old.
Database /home/grg/data/AmylFS must be updated because the database file is too
old.
ERROR No module named nuclearImaging.Interface_maker.GroupBoxCustom

--------------------------------------------------------------------------------
ImportError
processes.py (5042) in addDir:

toolbox=toolbox) # two arguments : process fullpath and category (directories
separated by /)

processes.py (4675) in readProcess:

moduleName, fileIn, fileName,